In [ ]:
# Use "numpy<2" until version compatibility is fixed
%pip install "numpy<2"
%pip install pandas
%pip install scanpy
%pip install igraph
%pip install leidenalg

import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [ ]:
outputdir_path = os.getenv('clustering_outdir')
if outputdir_path is None:
    outputdir_path = '/data/clustering_data_export/'

if not os.path.exists(outputdir_path):
    os.makedirs(outputdir_path)

# Make an output directory for generated graphs/plots,
# next to clustering output directory
plot_output_dir = os.path.join(Path(outputdir_path).parents[0], "cluster_plots")
if not os.path.exists(plot_output_dir):
    os.makedirs(plot_output_dir)


print(outputdir_path)
print(plot_output_dir)

In [ ]:
# Directory with segmentation data stored in 'segmentation_dir' env variable
segmentation_dir_path = os.getenv('segmentation_dir')
if segmentation_dir_path is None:
    segmentation_dir_path = '/data/segmentation_data/'

# Read all CSV files in the segmentation directory, store them in pandas DataFrames
segmentation_data = []
for filename in os.listdir(segmentation_dir_path):
    print(filename)
    if filename.endswith('cell-measurements.csv'):
        file_path = os.path.join(segmentation_dir_path, filename)
        df = pd.read_csv(file_path)
        segmentation_data.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(segmentation_data, ignore_index = True)

print(df.shape)
df.head()

In [ ]:
# Regions/annotations in segmentation data files
print(df['Parent'].unique())
#df.head()

In [ ]:
# If certain regions/annotations are processed, uncomment the line below and add to list
# df = df[df['Parent'] == 'Test Core']
df.head()

In [ ]:
import numpy as np
from scipy.stats import zscore

def normalize_data(data):
    cofactors = data.quantile(0.20)
    cofactors[cofactors <= 0] = 1e-6
    data_arcsinh = np.arcsinh(data / cofactors)
    data_zscored_biomarkers = data_arcsinh.apply(zscore, axis=0)
    data_zscored_cells = data_zscored_biomarkers.apply(zscore, axis=1)
    return data_zscored_cells

def normalize_cell_data(data):
    """
    Normalize cell data using three steps:
    1. Arcsinh transformation with cofactor equal to the 20th percentile of each biomarker.
    2. Z-standard normalization across all cells for each biomarker.
    3. Z-standard normalization across all biomarkers for each cell.

    Parameters:
    - data: pandas DataFrame where rows represent cells and columns represent biomarkers.

    Returns:
    - normalized_data: pandas DataFrame with normalized values.
    """
    # Step 1: Arcsinh transformation
    def arcsinh_transform(column):
        p20 = np.percentile(column, 20)
        if p20 > 0:
            return np.arcsinh(column / p20)
        else:
            return np.arcsinh(column)
    
    # Apply arcsinh transformation to each biomarker (column)
    arcsinh_data = data.apply(arcsinh_transform, axis=0)
    
    # Step 2: Z-standard normalization across all cells for each biomarker
    z_normalized_biomarker = arcsinh_data.apply(
        lambda col: (col - col.mean()) / col.std(), axis=0
    )
    
    # Step 3: Z-standard normalization across all biomarkers for each cell
    z_normalized_cells = z_normalized_biomarker.apply(
        lambda row: (row - row.mean()) / row.std(), axis=1
    )
    
    # Return the final normalized dataframe
    return z_normalized_cells

In [ ]:
# Save the annotations/regions for each cell in a list
# TODO: If there is a mismatch in the number of cells and regions, you may need to filter df['ROI'] to "Polygon" first
regions = df['Parent'].to_list()
print(len(regions))
# regions

In [ ]:

# List of exclude
excludeColumns = ["min", "max", "median", "std.dev", "area", "length", "circularity", "solidity", "diameter", "cluster"]
df_to_export = df[['Centroid X µm', 'Centroid Y µm', 'Image']]
data_cleaned = df.iloc[:, 11:]
na_rows_to_drop = data_cleaned[data_cleaned.isna().any(axis=1)].index

data_cleaned = data_cleaned.drop(index=na_rows_to_drop)
df_to_export = df_to_export.drop(index=na_rows_to_drop)

# Filter out columns that contain any of the  to exclude
data_cleaned = data_cleaned.loc[:, ~data_cleaned.columns.str.contains('|'.join(excludeColumns), case=False)]
print(data_cleaned.shape)
print(df_to_export.shape)

In [ ]:
# List of biomarkers to exclude
excludeBiomarkers = ['DAPI', 'CD123', 'Ki67', 'CD3e', 'CD8', 'Pan-Cytokeratin', 'CD107a', 'CD4', 'CD20', 'PCNA', 'CD141', 'aSMA', 'TCF7', 'CD138',
                     'Vimentin', 'Ecadherin', 'PD1', 'MHC I', 'PDL1', 'TOX', 'CD11b', 'GATA-3', 'HLA-DR', 'CD14', 'FoxP3', 'CD34', 'CD15',
                     'CD45RO', 'CXCR3', 'S100A8/9', 'Tim-3', 'GzmB', 'CD31', 'p16', 'SOX2', 'EpCAM', 'VISTA', 'CD66b', 'CXCR1', 'CD45',
                     'T-bet/TBX21', 'CD163', 'CD56', 'CD68', 'LAG-3', 'LEF-1', 'MPO', 'CD11c', 'IFNG', 'PDGFr', 'Galectin-3', 'Cathepsin-L',
                     'Podoplanin', 'H2A.X', 'GP100', 'CD27', 'T62', 'BX090', 'T55']
excludeBiomarkers = ["DAPI"]
# Filter out columns that contain any of the columns to exclude
if len(excludeBiomarkers) > 0:
    data_cleaned = data_cleaned.loc[:, ~data_cleaned.columns.str.contains('|'.join(excludeBiomarkers), case=False)]
print(data_cleaned.shape)
data_cleaned.head()

In [ ]:
data_cleaned = normalize_cell_data(data_cleaned)
print(data_cleaned.shape)
data_cleaned.head()

In [ ]:
# List of keywords for compartmental data
compartments = ['cell', 'membrane', 'cytoplasm', 'nucleus']

# Function to filter the data for each compartment
def filter_by_compartment(data, compartment):
    # Select columns that contain the specific keyword
    return data[[col for col in data.columns if compartment in col.lower()]]

# Separate data for each compartment
cell_data = filter_by_compartment(data_cleaned, 'cell')
membrane_data = filter_by_compartment(data_cleaned, 'membrane')
cytoplasm_data = filter_by_compartment(data_cleaned, 'cytoplasm')
nucleus_data = filter_by_compartment(data_cleaned, 'nucleus')

In [ ]:
print(cell_data.shape)
cell_data.head()

In [ ]:
print(membrane_data.shape)
membrane_data.head()

In [ ]:
print(nucleus_data.shape)
nucleus_data.head()

In [ ]:
print(cytoplasm_data.shape)
cytoplasm_data.head()

In [ ]:
# Run Leiden algorithm and plot UMAP for each compartment
def leiden_clustering_and_umap(compartment_data, compartment_name):
    # Create AnnData object from the compartment data
    adata = sc.AnnData(compartment_data.values)
    adata.var_names = compartment_data.columns
    
    # NEW: add regions to adata as new column
    adata.obs['region'] = regions[:adata.shape[0]] # Ensure the length matches the number of cells
    
    
    # Step 4: Normalize and preprocess the data (if required)
    # Usually for cell data, normalization is required, but you can adjust as needed
    # (Normalization is already done in the previous steps)
    #sc.pp.log1p(adata)  # Log-transforming the data to reduce skewness

    # Batch effect correction with ComBat, use 'region' as batch key
    # Useful for multiple regions and/or image sources
    sc.pp.combat(adata, key = 'region')
    
    # Step 5: Compute neighborhood graph
    sc.pp.neighbors(adata, n_neighbors=30)
    
    # Step 6: Run Leiden clustering
    sc.tl.leiden(adata, resolution=1.0)  # You can tweak the resolution parameter
    
    # Step 7: Visualize the clustering result using UMAP
    sc.tl.umap(adata, min_dist=0.0001)
    sc.pl.umap(adata, color='leiden', title=f'Leiden Clustering - {compartment_name}', show=False)
    
    ax = plt.gca()
    ax.set_xlabel('')
    ax.set_ylabel('')

    plt.savefig(os.path.join(plot_output_dir, "umap.png"), bbox_inches = 'tight')
    plt.show()
    
    return adata

In [ ]:
import scanpy as sc
# Run Leiden clustering and UMAP for each compartment
cell_adata = leiden_clustering_and_umap(cell_data, 'Cell')
#membrane_adata = leiden_clustering_and_umap(membrane_data, 'Membrane')
#cytoplasm_adata = leiden_clustering_and_umap(cytoplasm_data, 'Cytoplasm')
#nucleus_adata = leiden_clustering_and_umap(nucleus_data, 'Nucleus')

In [ ]:
cell_adata

In [ ]:
cell_adata.obs['leiden']

In [ ]:
expression_df = pd.DataFrame(cell_adata.X, index=cell_adata.obs_names, columns=cell_adata.var_names)
clusters_data = expression_df.copy()
clusters_data['leiden'] = cell_adata.obs['leiden']

clusters_data

In [ ]:
cluster_expresssion_mean = clusters_data.groupby('leiden').mean()
cluster_expresssion_mean

%pip install openpyxl
cluster_expresssion_mean.to_excel(os.path.join(plot_output_dir, "cluster_means.xlsx"))

In [ ]:
def clustering_heatmap(compartment_data, compartment_name):
    #pio.renderers.default = 'iframe'
    # Sample data
    clusters = compartment_data.obs['leiden']
    
    # Extract expression data as a DataFrame
    expression_data = pd.DataFrame(compartment_data.X, index=compartment_data.obs_names, columns=compartment_data.var_names)
    
    # Combine clusters with expression data
    data_with_clusters = expression_data.copy()
    data_with_clusters['leiden'] = clusters
    
    # Group by clusters and calculate the mean expression per cluster
    mean_expression_per_cluster = data_with_clusters.groupby('leiden').mean()
    heatmap_df = mean_expression_per_cluster.T
    
    clustergram = dashbio.Clustergram(
        data=heatmap_df,
        cluster='all',
        column_labels=list(heatmap_df.columns.values),
        row_labels=list(heatmap_df.index.str.replace(f': {compartment_name}: Mean', '', regex=False)),
        color_map='Jet',
        #color_continuous_scale='Jet',
        #color_map=[[0.1, '#971D2B'], [0.2, '#C74637'], [.3, '#E58256'], [.4, '#F3C17E'], [.5, '#FBEEAE'],[.6, '#EFF7DF'], [.7, '#C3E0EB'], [.8, '#8CB5D3'], [.9, '#5779B2'], [1.0, '#323690']],  # Color scale
        height=1200,  # Height of the clustergram
        width=1000,  # Width of the clustergram,
        #display_ratio=[0.4, 0.6]
    )
    
    clustergram.for_each_trace(
        lambda t: t.update(hovertemplate="Cluster: %{x}<br>Biomarker: %{y}<br>Value: %{z}")
        if isinstance(t, go.Heatmap)
        else t
    )

    return clustergram

In [ ]:
%pip install dash
%pip install dash_bio
%pip install dash-renderer
%pip install dash_html_components
%pip install dash_core_components
%pip install kaleido
import kaleido
import plotly
import plotly.offline as py
import pandas as pd
import numpy as np
import plotly.express as px
import dash_bio as dashbio
import plotly.graph_objects as go

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

#py.init_notebook_mode()


import plotly.io as pio
pio.renderers.default = 'iframe'

In [ ]:
from IPython.display import display
clustergram = clustering_heatmap(cell_adata, 'Cell')
fig = go.Figure(clustergram)
heatmap_location = os.path.join(plot_output_dir, "heatmap.html")
fig.write_html(heatmap_location)
display(clustergram)  # Explicitly display it#clustergram_nucleus = clustering_heatmap(nucleus_adata, 'Nucleus')
#display(clustergram_nucleus)
#clustergram_cytoplasm = clustering_heatmap(cytoplasm_adata, 'Cytoplasm')
#display(clustergram_cytoplasm)
#clustergram_membrane = clustering_heatmap(membrane_adata, 'Membrane')
#display(clustergram_membrane)

In [ ]:
print(df_to_export.shape)

In [ ]:
df_to_export['Leiden Cluster Label'] =  cell_adata.obs['leiden'].values
#print(cell_adata.uns['leiden_colors'])
cluster_colors = cell_adata.uns['leiden_colors']
df_to_export['cluster_color'] = df_to_export['Leiden Cluster Label'].map(lambda x: cluster_colors[int(x)])
print(df_to_export.head())
df_to_export.to_csv("".join([outputdir_path, '/leiden_clustering_export.csv']), index=False)